In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Imports and TPU setting

In [1]:
! pip uninstall kaggle -q
! pip install kaggle -q
! pip install transformers -q

Proceed (y/n)? y


In [2]:
import os
import re
import time
import tokenizers
import numpy as np
import pandas as pd
import transformers
from tqdm import tqdm
import tensorflow as tf
from google.colab import files
import tensorflow_datasets as tfds
import tensorflow.keras.backend as K
from transformers import BertTokenizer
from transformers import TFRobertaModel
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
%matplotlib inline

tf.get_logger().setLevel('ERROR')

#### Load the data

In [3]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'tweet-sentiment-extraction'

Saving kaggle.json to kaggle.json
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 101MB/s]


In [4]:
!unzip '/content/tweet-sentiment-extraction.zip'

Archive:  /content/tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
train = pd.read_csv('/content/train.csv')
train['text'] = train['text'].astype(str)
train['selected_text'] = train['selected_text'].astype(str)

test = pd.read_csv('/content/test.csv')
test['text'] = test['text'].astype(str)

In [4]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


#### Text augmentation

In [5]:
!pip install emoji -qq
!pip install googletrans -qq
from text import *

In [6]:
augmentation = textAugmentation()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [29]:
augmentation.get_most_similar('bday')

[('b-day', 0.9375146627426147),
 ('birthday', 0.9229019284248352),
 ('happy', 0.872036874294281),
 ('luv', 0.8638796806335449),
 ('nite', 0.8542247414588928)]

In [10]:
augmentation.get_MLM('This is <mask> cool')

[{'score': 0.5451372265815735,
  'sequence': '<s>This is pretty cool</s>',
  'token': 1256,
  'token_str': 'Ġpretty'},
 {'score': 0.14360912144184113,
  'sequence': '<s>This is really cool</s>',
  'token': 269,
  'token_str': 'Ġreally'},
 {'score': 0.07481756806373596,
  'sequence': '<s>This is super cool</s>',
  'token': 2422,
  'token_str': 'Ġsuper'},
 {'score': 0.05064457654953003,
  'sequence': '<s>This is very cool</s>',
  'token': 182,
  'token_str': 'Ġvery'},
 {'score': 0.022713985294103622,
  'sequence': '<s>This is kinda cool</s>',
  'token': 24282,
  'token_str': 'Ġkinda'}]

In [14]:
augmentation.get_synonym('food')

['nutrient', 'solid_food', 'food_for_thought', 'intellectual_nourishment']

In [19]:
augmentation.get_translation('school', 'fr')

'école'

#### Preprocess

In [ ]:
print(f"train shape: {train.shape} \ntest shape: {test.shape}")
print(f"ratio between lables in test: {test.sentiment.value_counts()[1] / test.sentiment.value_counts()[0]}")
print("-"*4)
mean_word_len = train.text.apply(lambda x: len(x.split(" "))).mean()
print(f"Dataset with shape of {train.shape[0]} samples. \nMean number of words is: {mean_word_len}. \nDistribution of lables is: \n{train.sentiment.value_counts()}")

train shape: (27481, 4) 
test shape: (3534, 3)
ratio between lables in test: 0.7713286713286713
----
Dataset with shape of 27481 samples. 
Mean number of words is: 13.7794476183545. 
Distribution of lables is: 
neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64


#### Modelling

In [ ]:
MAX_LEN = 96

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", lowercase = True)

In [ ]:
# path = "/content/drive/My Drive/projects/tweet sentiment extraction"

# tokenizer = tokenizers.ByteLevelBPETokenizer(
#     vocab_file=path + os.sep + 'vocab.json', 
#     merges_file=path + os.sep + 'merges.txt', 
#     lowercase=True,
#     add_prefix_space=True
# )

In [ ]:
sentiment_id = {'positive': tokenizer.encode_plus('positive', add_prefix_space=True, max_length = MAX_LEN, pad_to_max_length = True, return_attention_mask = True, return_token_type_ids = True, truncation=True).input_ids[1], 
                'negative': tokenizer.encode_plus('negative', add_prefix_space=True, max_length = MAX_LEN, pad_to_max_length = True, return_attention_mask = True, return_token_type_ids = True, truncation=True).input_ids[1], 
                'neutral':  tokenizer.encode_plus('neutral', add_prefix_space=True, max_length = MAX_LEN, pad_to_max_length = True, return_attention_mask = True, return_token_type_ids = True, truncation=True).input_ids[1]}
sentiment_id

{'negative': 2430, 'neutral': 7974, 'positive': 1313}

In [ ]:
for i in range(4):
  print(f"Special token {i} -> {tokenizer.decode([i])}")

Special token 0 -> <s>
Special token 1 -> <pad>
Special token 2 -> </s>
Special token 3 -> <unk>


In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
ct = train.shape[0]

input_ids = np.ones((ct,MAX_LEN),dtype='int32')

attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')

start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in tqdm(range(train.shape[0])):
    text1 = " " + " ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())

    enc = tokenizer.encode_plus(text1.lower(), 
                       add_prefix_space=True,
                       max_length = MAX_LEN, # max length of the text that can go to BERT
                       pad_to_max_length = True, # add [PAD] tokens
                       return_attention_mask = True, # add attention mask to not focus on pad tokens
                       return_token_type_ids = True,
                       truncation=True)
    
    input_ids[k] = np.array(enc.input_ids)
    input_ids[k, np.where(input_ids[k] == 2)[0][0]+1 : np.where(input_ids[k] == 2)[0][0]+4] = 2
    input_ids[k, np.where(input_ids[k] == 2)[0][0]+2] = sentiment_id[train.loc[k,'sentiment']]

    attention_mask[k] = np.array(enc.attention_mask)
    attention_mask[k, np.where(attention_mask[k] == 1)[0][-1] + 1 : np.where(attention_mask[k] == 1)[0][-1] + 4] = 1
   
    token_type_ids[k] = np.array(enc.token_type_ids)

    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)] = 1

    if text1[idx-1] == ' ': 
        chars[idx-1] = 1 
        
    idx=0
    toks = []
    for i, t in enumerate(enc.input_ids[1:]):
          w = tokenizer.decode([t])
          if np.sum(chars[idx:idx+len(w)]) > 0: # if we are in overlapp location append the token
              toks.append(i)
          idx += len(w)
    
    if len(toks) > 0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

100%|██████████| 27481/27481 [00:39<00:00, 688.10it/s]


In [ ]:
ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in tqdm(range(test.shape[0])):
        
    text1 = " " + " ".join(test.loc[k,'text'].split())

    enc = tokenizer.encode_plus(text1.lower(), 
                       add_prefix_space=True,
                       max_length = MAX_LEN, # max length of the text that can go to BERT
                       pad_to_max_length = True, # add [PAD] tokens
                       return_attention_mask = True, # add attention mask to not focus on pad tokens
                       return_token_type_ids = True,
                       truncation=True)
    
    input_ids_t[k] = np.array(enc.input_ids)
    input_ids_t[k, np.where(input_ids_t[k] == 2)[0][0]+1 : np.where(input_ids_t[k] == 2)[0][0]+4] = 2
    input_ids_t[k, np.where(input_ids_t[k] == 2)[0][0]+2] = sentiment_id[test.loc[k,'sentiment']]

    attention_mask_t[k] = np.array(enc.attention_mask)
    attention_mask_t[k, np.where(attention_mask_t[k] == 1)[0][-1] + 1 : np.where(attention_mask_t[k] == 1)[0][-1] + 4] = 1
   
    token_type_ids_t[k] = np.array(enc.token_type_ids)

100%|██████████| 3534/3534 [00:01<00:00, 2632.55it/s]


###### Build model

In [ ]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    roberta_model = TFRobertaModel.from_pretrained('roberta-base')
    x = roberta_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [ ]:
K.clear_session()
model = build_model()
model.summary()

Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 96, 768), (N 124645632   input_1[0][0]                    
                                                                 input_2[0][0]         

#### Training

In [ ]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

from sklearn.model_selection import train_test_split
idxT,idxV = train_test_split(list(range(input_ids.shape[0])), test_size=0.2, random_state=42)
    
K.clear_session()
model = build_model()

my_callbacks = [
                tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='min', baseline=None, restore_best_weights=True)
]

model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
    epochs=3, batch_size=32, verbose=DISPLAY, callbacks=my_callbacks,
    validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
    [start_tokens[idxV,], end_tokens[idxV,]]))

print('#'*5)
print("Predicting Validation")
print('#'*5)
oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)

# DISPLAY FOLD JACCARD
all = []
for k in idxV:
    a = np.argmax(oof_start[k,])
    b = np.argmax(oof_end[k,])
    if a>b: 
        st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
    else:            
        st = tokenizer.decode(input_ids[k][a:b+1])
    all.append(jaccard(st,train.loc[k,'selected_text']))

Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
687/687 [==============================] - 272s 396ms/step - loss: 2.2786 - activation_loss: 1.0642 - activation_1_loss: 1.2144 - val_loss: 2.3981 - val_activation_loss: 0.9355 - val_activation_1_loss: 1.4626
Epoch 2/3
687/687 [==============================] - 270s 393ms/step - loss: 1.7777 - activation_loss: 0.8702 - activation_1_loss: 0.9075 - val_loss: 1.7080 - val_activation_loss: 0.8194 - val_activation_1_loss: 0.8886
Epoch 3/3
687/687 [==============================] - 270s 392ms/step - loss: 1.6127 - activation_loss: 0.8082 - activation_1_loss: 0.8046 - val_loss: 1.6273 - val_activation_loss: 0.8100 - val_activation_1_loss: 0.8173
#####
Predicting Validation
#####
172/172 [==============================] - 23s 134ms/step


In [ ]:
print(f"Jaccard = {np.mean(all)}")

Jaccard = 0.6972039501794071
